In [ ]:
import math
from risq import *

### Data (survivor group)

In [ ]:
survivor_group_distribution = [(age - 5, cases / 100_000) for (age, cases) in [
    (22, 5),
    (27, 10),
    (32, 25),
    (37, 98),
    (42, 269),
    (47, 337),
    (52, 360),
    (57, 358),
    (62, 343),
    (67, 319),
    (72, 288),
    (77, 250),
    (82, 206),
    (87, 158),
]]

### Parameter fitting

In [ ]:
def loss_combined_model(
    special_prob_mutate: float,
    special_prob_dying: float,
    special_prob_spread: float,
    *,
    verbose: bool = False
) -> float:
    # Convert probabilities using math.erf (for better optimization behaviour)
    special_prob_mutate = (1.0 + math.erf(special_prob_mutate)) / 2.0
    special_prob_dying = (1.0 + math.erf(special_prob_dying)) / 2.0
    special_prob_spread = (
        1.0 + math.erf(special_prob_spread)) / 2.0

    # Create special model
    model_special = create_six_mutations_model(
        prob_mutate=special_prob_mutate,
        prob_dying=special_prob_dying,
        prob_spread=special_prob_spread
    )

    # Combine with default model (from control group)
    model_default = create_six_mutations_model(
        prob_mutate = 0.020360638820134225,
        prob_dying  = 0.5647743182095744,
        prob_spread = 0.01593379960021246
    )
    model = CombinedModel(model_default, model_special,
                          lambda time: time >= 30 * 12 and time < 30 * 12 + 6)

    simulation = NeighboringCellMethod(model)

    # Assumption: number of cells
    num_cells = 1_000_000

    # Compute loss as total square difference of probabilities
    loss = 0.0
    prev_prob_cdf = 0.0
    for age, prob in survivor_group_distribution:
        prob_cdf = compute_cancer_probability(
            method=simulation,
            num_cells=num_cells,
            time=age * 12,  # 1 time step = 1 month
            state_cancer=7  # C
        )
        prob_est = prob_cdf - prev_prob_cdf
        loss += (prob_est - prob) ** 2

        if verbose:
            print(f"model({age}) = {prob_est} (expected {prob})")

        prev_prob_cdf = prob_cdf


    if verbose:
        print()
        print(f"special_prob_mutate = {special_prob_mutate}")
        print(f"special_prob_dying  = {special_prob_dying}")
        print(f"special_prob_spread = {special_prob_spread}")
        print()
        print(f"loss = {loss}")

    return loss

In [ ]:
# Optimize parameters for model
if 'params_combined' not in locals():
    params_combined = {
        # Parameters for 6 month exposure
        'special_prob_mutate': -1.4224279888746723,
        'special_prob_dying': -0.3930155992176092,
        'special_prob_spread': -4.2114527573257465
        # loss = 8.60e-6

        # Parameters for 1 month exposure
        # 'special_prob_mutate': -1.154471196332396,
        # 'special_prob_dying': -0.9441877931455833,
        # 'special_prob_spread': -4.739466242699623
        # loss = 8.82e-6
    }

params_combined = steepest_descent(
    loss_combined_model,
    params_combined,
    num_iter=50,
    delta=0.001
)

print()
print(f"┌────────────────────┐")
print(f"│ PRINTING ESTIMATES │")
print(f"└────────────────────┘")
loss_combined_model(**params_combined, verbose=True);

In [ ]:
model_special = create_six_mutations_model(
    # Parameters for 6 months exposure
    prob_mutate = 0.022130165443877803,
    prob_dying  = 0.2891710611970556,
    prob_spread = 1.2932728221670686e-09

    # Parameters for 1 month exposure
    # prob_mutate = 0.05126933392798483,
    # prob_dying  = 0.09089184036293568,
    # prob_spread = 1.0235923220136556e-11
)

model_default = create_six_mutations_model(
    prob_mutate = 0.020360638820134225,
    prob_dying  = 0.5647743182095744,
    prob_spread = 0.01593379960021246
)

model = CombinedModel(
    model_default,
    model_special,
    lambda time: time >= 30 * 12 and time < 30 * 12 + 6
)

print_latex_table(
    model,
    method = NeighboringCellMethod,
    num_cells = 1_000_000,
    state_cancer = 7,
    distribution = survivor_group_distribution
)